# The Vascular Modeling Toolkit
## Pypes Tutorial

### Overview

PypeS is a small high-level framework of Python pipeable scripts.

### Motivation

Writing classes implementing algorithms and writing actual tools to be used for everyday work are two distinct tasks. Very often a well-designed object-oriented library ends up to be used in ever-growing collections of shell, Python or Tcl scripts or small C programs, each with its own argument parsing and I/O sections. Very often high-level code is duplicated to provide slightly different functionality. On the other side, writing a GUI is a time-consuming task, and adding new functionality requires time, which might deter experimentation. PypeS goes in the direction of providing a flexible framework for high-level code, both from the user's and from the developer's points of view. The user wants to get things done minimizing the work required and the amount of intermediate data generated. The coder wants to limit the amount of code, she/he has to cut and paste (and maintain), and to quickly add new functionality and make it interact with what she/he's ever written before.

### Description

PypeS is basically made up of two main classes:

- PypeScript: the base class for every high-level script. It manages parsing, instantiates proper input/output methods for the script and keeps the script structure consistent
- Pype: the class that orchestrates the interaction among PypeScripts. It enables one to pipe one PypeScript after another, and it takes care of connecting the right arguments from one script to the other. It can be called from the command line by issuing 
        pype pype-arguments 
or automatically instantiated from a pypescript. 

Each PypeScript is at the same time:

- a script which can be called from the command line and piped to other scripts
- a class which can be called from Python code (e.g. inside a tkinter GUI or JupyterNotebook) 

### Historic Methods of Interacting with Pypes

- [vmtk Scripts Tutorial](http://www.vmtk.org/tutorials/ScriptsBasic.html)
- [Basic PypeS tutorial](http://www.vmtk.org/tutorials/PypesBasic.html)

### Using VMTK Within Interactive Pypes Sessions
VMTK 2.0 provides seamless integration of vmtk pypes within the interactive Jupyter Notebook interpreter. By using the new `vmtk.run()` interface, the power of pypes has been extended, allowing for easy access and specification to vmtk data objects within an interactive python session. 

For those familiar with the pypes interface, there are no backwards incompatible API changes. 

To begin to use the new interface, simply `import vmtk`

In [ ]:
import vmtk

With this simple import, we have access to not only `vmtkscripts` but to a new method symbol which has been exported (the `run` method).

In [ ]:
vmtk.__all__

We can specify a pype by setting an argument string into the `vmtk.run` interface just as a normal pype would work. The notebook will display all the normal logging data, and any visualizations will appear in a seperate render window

In [ ]:
imagePype = vmtk.run('vmtkimagereader -ifile ./aorta.mha --pipe vmtkimageviewer -windowlevel 0.0 500')

![vmtk imagePype](./imageViewerPype.png)

### Accessing Pype Members from vmtk.run()

By setting the name `imagePype` to our vmtk.run method, we now have access to every input/output member for every script specified within the pype.

If we wanted to access to vtkImageData which is used to store our image while it's in memory, we can simply assign it to:

In [ ]:
aortaImage = imagePype.vmtkimagereader.OutputMembers.Image

We can inspect the vtkImageData object assigned to the python variable by just printing it

In [ ]:
print(aortaImage)

likewise, we can access properties of the render window (even after it has closed out of processing)

In [ ]:
windowLevel = imagePype.vmtkimageviewer.InputMembers.WindowLevel

In [ ]:
windowLevel

we can view the image in a new pype with the new vmtk.run interface. 

Just specify a variable name within `{curly braces}` of the input string, and the object will be seamlessly placed as in input into the new pype instance.

In [ ]:
imageVolumeViewer = vmtk.run('vmtkimagevolumeviewer -i {aortaImage} -preset CT-AAA2')

![imageVolumeViewerPype](./imageVolumeViewer.png)

### More Examples assigning variables to a pype

In [ ]:
fileName = './aorta.mha'
flip = [0, 1, 1]
level = 700

In [ ]:
surfaceExtractorPype = vmtk.run('vmtkimagereader -ifile {fileName} -flip {flip} --pipe vmtkmarchingcubes -l {level} --pipe vmtksurfaceviewer')

![surfaceViewerPype](./surfaceViewerPype.png)

Variables defined within a pype can be passed as arguments to another pype by simply enclosing them in `{curly braces}`

In [ ]:
myargs = 'vmtkrenderer --pipe vmtksurfaceviewer -i {surfaceExtractorPype.vmtkmarchingcubes.OutputMembers.Surface} --pipe vmtkimageviewer -i {imagePype.vmtkimagereader.OutputMembers.Image}'

In [ ]:
multipleRendererPype = vmtk.run(myargs)

![multipleRendererPype](./multipleRendererPype.png)